In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/huggingface/transformers
%cd transformers
!pip install .
!pip install seqeval
%cd ..

Cloning into 'transformers'...
remote: Enumerating objects: 191301, done.
remote: Counting objects: 100% (22585/22585), done.
remote: Compressing objects: 100% (1789/1789), done.
remote: Total 191301 (delta 22063), reused 20797 (delta 20796), pack-reused 168716
Receiving objects: 100% (191301/191301), 199.25 MiB | 15.27 MiB/s, done.
Resolving deltas: 100% (136324/136324), done.
Updating files: 100% (4135/4135), done.
/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8680419 sha256=32cd2504d7919ad6d42f2d672b7e6bd03a6faffd1fae90c885e2be156c0b8731
  Stored in directory: /tmp/pip-ephem-wheel-cache-5awobez1/wheels/7c/35/80/e946b22a081210c6642e607ed65b2a5b9a4d9259695ee2caf5
Successfully built transformers
  Attempting uninstall: transformers
    Found existing insta

# Dataset overview and Preprocessing

**Check dataset size**

In [ ]:
#!wc -l /content/drive/MyDrive/NER-BERT/Conll2003/train.txt
#!wc -l /content/drive/MyDrive/NER-BERT/Conll2003/test.txt
#!wc -l /content/drive/MyDrive/NER-BERT/Conll2003/valid.txt

!wc -l /content/drive/MyDrive/NER-BERT/data/pd-bc5/pd-bc5.train
!wc -l /content/drive/MyDrive/NER-BERT/data/pd-bc5/pd-bc5.test
!wc -l /content/drive/MyDrive/NER-BERT/data/pd-bc5/pd-bc5.dev

144186 /content/drive/MyDrive/NER-BERT/data/pd-bc5/pd-bc5.train
124420 /content/drive/MyDrive/NER-BERT/data/pd-bc5/pd-bc5.test
123869 /content/drive/MyDrive/NER-BERT/data/pd-bc5/pd-bc5.dev


**First 20 lines of train dataset**

In [ ]:
!head -n20 /content/drive/MyDrive/NER-BERT/data/pd-bc5/pd-bc5.train

studies O
on O
magnesiums O
mechanism O
of O
action O
in O
digitalis B-Plant
induced O
arrhythmias B-Disease

the O
mechanism O
by O
which O
magnesium O
affects O
digitalis B-Plant
induced O
arrhythmias B-Disease


**Keep only words and name entity tags**

In [4]:
%cd /content/drive/MyDrive/NER-BERT/data/Multi-Source/S800-Plantblog-Textberg-PDR-BC5CDR

/content/drive/MyDrive/NER-BERT/data/Multi-Source/S800-Plantblog-Textberg-PDR-BC5CDR


In [ ]:
!cat /content/drive/MyDrive/NER-BERT/data/pd-ncbi/pd-ncbi.train | cut -d " " -f 1,2 > pdncbi-train_temp.txt
!cat /content/drive/MyDrive/NER-BERT/data/pd-ncbi/pd-ncbi.test | cut -d " " -f 1,2 > pdncbi-test_temp.txt
!cat /content/drive/MyDrive/NER-BERT/data/pd-ncbi/pd-ncbi.dev| cut -d " " -f 1,2 > pdncbi-dev_temp.txt

**Run preprocess script**

In [5]:
!cp /content/drive/MyDrive/NER-BERT/preprocess.py /content/drive/MyDrive/NER-BERT/data/Multi-Source/S800-Plantblog-Textberg-PDR-BC5CDR/preprocess.py

In [6]:
#!python3 preprocess.py pd-ncbi.train "bert-base-multilingual-cased" 128 > ready/train.txt
#!python3 preprocess.py pd-ncbi.test "bert-base-multilingual-cased" 128 > ready/test.txt
#!python3 preprocess.py pd-ncbi.dev "bert-base-multilingual-cased" 128 > ready/dev.txt

!python3 preprocess.py pst-pdr-bc5cdr.train "dmis-lab/bern2-ner" 128 > bern2-ner/train.txt
!python3 preprocess.py pst-pdr-bc5cdr.test "dmis-lab/bern2-ner" 128 > bern2-ner/test.txt
!python3 preprocess.py pst-pdr-bc5cdr.dev "dmis-lab/bern2-ner" 128 > bern2-ner/dev.txt


#biobert-base-cased-v1.2-finetuned-ner

tokenizer_config.json: 100% 62.0/62.0 [00:00<00:00, 343kB/s]
config.json: 100% 633/633 [00:00<00:00, 3.31MB/s]
vocab.json: 100% 912k/912k [00:00<00:00, 3.74MB/s]
merges.txt: 100% 472k/472k [00:00<00:00, 1.90MB/s]
added_tokens.json: 100% 17.0/17.0 [00:00<00:00, 66.3kB/s]
special_tokens_map.json: 100% 772/772 [00:00<00:00, 4.29MB/s]


In [7]:
%cd /content/drive/MyDrive/NER-BERT/data/Multi-Source/S800-Plantblog-Textberg-PDR-BC5CDR/bern2-ner

/content/drive/MyDrive/NER-BERT/data/Multi-Source/S800-Plantblog-Textberg-PDR-BC5CDR/bern2-ner


In [10]:
!cat train.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels-train.txt

# Train section

In [11]:
!cp /content/drive/MyDrive/NER-BERT/run_ner.py /content/drive/MyDrive/NER-BERT/data/Multi-Source/S800-Plantblog-Textberg-PDR-BC5CDR/bern2-ner/run_ner.py
!cp /content/drive/MyDrive/NER-BERT/utils_ner.py /content/drive/MyDrive/NER-BERT/data/Multi-Source/S800-Plantblog-Textberg-PDR-BC5CDR/bern2-ner/utils_ner.py

In [ ]:
!python3 run_ner.py \
  --data_dir /content/drive/MyDrive/NER-BERT/data/Multi-Source/S800-Plantblog-Textberg-PDR-BC5CDR/bern2-ner \
  --model_type bert \
  --labels /content/drive/MyDrive/NER-BERT/data/Multi-Source/S800-Plantblog-Textberg-PDR-BC5CDR/bern2-ner/labels.txt \
  --model_name_or_path "dmis-lab/biobert-v1.1" \
  --output_dir "/content/drive/MyDrive/NER-BERT/model/pst-pdr-bc5cdr/bern2-ner" \
  --max_seq_length  128 \
  --num_train_epochs 10 \
  --per_gpu_train_batch_size 32 \
  --save_steps 1000 \
  --logging_steps 1000 \
  --seed 43 \
  --do_train \
  --do_eval \
  --do_predict \
  --overwrite_output_dir

2024-03-15 01:28:34.556195: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-15 01:28:34.556259: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-15 01:28:34.557750: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-15 01:28:35.780614: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
03/15/2024 01:28:37 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
03/15/2024 01:28:39 - INFO - __main__ -   Tokenizer arguments: {'do_lower_case': False}
pytorch_model

In [ ]:
def read_examples_from_file(file_path):
    """Read words and labels from a Multi-source data file.

    Args:
      file_path (str): path to NER data file.

    Returns:
      examples (dict): a dictionary with two keys: words (list of lists)
        holding words in each sequence, and labels (list of lists) holding
        corresponding labels.
    """
    with open(file_path, encoding="utf-8") as f:
        examples = {"words": [], "labels": []}
        words = []
        labels = []
        for line in f:
            if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                if words:
                    examples["words"].append(words)
                    examples["labels"].append(labels)
                    words = []
                    labels = []
            else:
                splits = line.split(" ")
                words.append(splits[0])
                if len(splits) > 1:
                    labels.append(splits[-1].replace("\n", ""))
                else:
                    # Examples could have no label for mode = "test"
                    labels.append("O")
    return examples

In [ ]:
y_true = read_examples_from_file("/content/drive/MyDrive/NER-BERT/data/Multi-Source/S800-Plantblog-Textberg-BC5CDR/biobert/test.txt")["labels"]
y_pred = read_examples_from_file("/content/drive/MyDrive/NER-BERT/model/pst-bc5cdr/biobert/test_predictions.txt")["labels"]

In [ ]:
from seqeval.metrics import classification_report as classification_report_seqeval

print(classification_report_seqeval(y_true, y_pred))

              precision    recall  f1-score   support

     Disease       0.74      0.74      0.74       101
       Plant       0.69      0.62      0.65        86

   micro avg       0.72      0.68      0.70       187
   macro avg       0.71      0.68      0.69       187
weighted avg       0.71      0.68      0.70       187



In [ ]:
import numpy as np
from sklearn.metrics import classification_report

print(classification_report(np.concatenate(y_true), np.concatenate(y_pred)))

              precision    recall  f1-score   support

   B-Disease       0.82      0.78      0.80       101
     B-Plant       0.81      0.73      0.77        85
   I-Disease       0.67      0.81      0.73        52
     I-Plant       0.94      0.59      0.72        29
           O       0.96      0.97      0.97      1307

    accuracy                           0.93      1574
   macro avg       0.84      0.78      0.80      1574
weighted avg       0.94      0.93      0.93      1574



## **Plant-Disease | NCBI | BERT**

In [ ]:
from seqeval.metrics import classification_report as classification_report_seqeval

print(classification_report_seqeval(y_true, y_pred))

              precision    recall  f1-score   support

     Disease       0.84      0.85      0.84      1060
       Plant       0.80      0.63      0.71        87

   micro avg       0.84      0.83      0.84      1147
   macro avg       0.82      0.74      0.78      1147
weighted avg       0.84      0.83      0.83      1147



In [ ]:
import numpy as np
from sklearn.metrics import classification_report

print(classification_report(np.concatenate(y_true), np.concatenate(y_pred)))

              precision    recall  f1-score   support

   B-Disease       0.87      0.86      0.86      1060
     B-Plant       0.90      0.70      0.78        86
   I-Disease       0.87      0.90      0.88      1137
     I-Plant       0.91      0.72      0.81        29
           O       0.99      0.99      0.99     23767

    accuracy                           0.98     26079
   macro avg       0.91      0.83      0.87     26079
weighted avg       0.98      0.98      0.98     26079



## **Plant-Disease | NCBI | BioBERT | Clean**

In [ ]:
from seqeval.metrics import classification_report as classification_report_seqeval

print(classification_report_seqeval(y_true, y_pred))

              precision    recall  f1-score   support

     Disease       0.80      0.89      0.84      1060
       Plant       0.84      0.80      0.82        87

   micro avg       0.80      0.88      0.84      1147
   macro avg       0.82      0.85      0.83      1147
weighted avg       0.81      0.88      0.84      1147



In [ ]:
import numpy as np
from sklearn.metrics import classification_report

print(classification_report(np.concatenate(y_true), np.concatenate(y_pred)))

              precision    recall  f1-score   support

   B-Disease       0.84      0.91      0.88      1060
     B-Plant       0.90      0.87      0.89        86
   I-Disease       0.86      0.93      0.89      1137
     I-Plant       0.88      0.76      0.81        29
           O       0.99      0.99      0.99     23767

    accuracy                           0.98     26079
   macro avg       0.90      0.89      0.89     26079
weighted avg       0.98      0.98      0.98     26079



## **Plant-Disease | BC5CDR | BERT**

---



In [ ]:
y_true = read_examples_from_file("/content/drive/MyDrive/NER-BERT/data/pd-bc5/biobert/test.txt")["labels"]
y_pred = read_examples_from_file("/content/drive/MyDrive/NER-BERT/model/pd-bc5/biobert/test_predictions.txt")["labels"]

In [ ]:
from seqeval.metrics import classification_report as classification_report_seqeval

print(classification_report_seqeval(y_true, y_pred))

              precision    recall  f1-score   support

     Disease       0.83      0.85      0.84      4284
       Plant       0.55      0.71      0.62        87

   micro avg       0.82      0.84      0.83      4371
   macro avg       0.69      0.78      0.73      4371
weighted avg       0.82      0.84      0.83      4371



In [ ]:
import numpy as np
from sklearn.metrics import classification_report

print(classification_report(np.concatenate(y_true), np.concatenate(y_pred)))


              precision    recall  f1-score   support

   B-Disease       0.88      0.88      0.88      4284
     B-Plant       0.61      0.80      0.69        86
   I-Disease       0.86      0.75      0.80      2971
     I-Plant       0.61      0.66      0.63        29
           O       0.99      0.99      0.99    112393

    accuracy                           0.98    119763
   macro avg       0.79      0.82      0.80    119763
weighted avg       0.98      0.98      0.98    119763



## **Plant-Disease | BC5CDR | BioBERT**

In [ ]:
y_true = read_examples_from_file("/content/drive/MyDrive/NER-BERT/data/pd-bc5/ready/test.txt")["labels"]
y_pred = read_examples_from_file("/content/drive/MyDrive/NER-BERT/model/pd-bc5/test_predictions.txt")["labels"]

In [ ]:
from seqeval.metrics import classification_report as classification_report_seqeval

print(classification_report_seqeval(y_true, y_pred))

              precision    recall  f1-score   support

     Disease       0.83      0.79      0.81      4285
       Plant       0.62      0.63      0.62        86

   micro avg       0.82      0.78      0.80      4371
   macro avg       0.72      0.71      0.72      4371
weighted avg       0.83      0.78      0.80      4371



In [ ]:
import numpy as np
from sklearn.metrics import classification_report

print(classification_report(np.concatenate(y_true), np.concatenate(y_pred)))

              precision    recall  f1-score   support

   B-Disease       0.87      0.81      0.84      4284
     B-Plant       0.70      0.71      0.70        85
   I-Disease       0.85      0.69      0.76      2971
     I-Plant       0.74      0.69      0.71        29
           O       0.99      0.99      0.99    112394

    accuracy                           0.98    119763
   macro avg       0.83      0.78      0.80    119763
weighted avg       0.98      0.98      0.98    119763



## **BC5CDR - Plantblog - S100 - Textberg | BioBERT**

In [ ]:
from seqeval.metrics import classification_report as classification_report_seqeval

print(classification_report_seqeval(y_true, y_pred))

              precision    recall  f1-score   support

     Disease       0.86      0.87      0.87      1174
       Plant       0.81      0.88      0.84       283

   micro avg       0.85      0.87      0.86      1457
   macro avg       0.84      0.87      0.85      1457
weighted avg       0.85      0.87      0.86      1457



In [ ]:
import numpy as np
from sklearn.metrics import classification_report

print(classification_report(np.concatenate(y_true), np.concatenate(y_pred)))

              precision    recall  f1-score   support

   B-Disease       0.90      0.89      0.89      1174
     B-Plant       0.86      0.91      0.89       281
   I-Disease       0.86      0.85      0.86       746
     I-Plant       0.86      0.93      0.89       215
           O       0.99      0.99      0.99     23419

    accuracy                           0.98     25835
   macro avg       0.89      0.91      0.90     25835
weighted avg       0.98      0.98      0.98     25835



# **Train: BC5CDR - Plantblog - S100 - Textberg | Test: Plant-Disease**


In [ ]:
y_true = read_examples_from_file("/content/drive/MyDrive/NER-BERT/data/Multi-Source/S800-Plantblog-Textberg-BC5CDR/biobert/test.txt")["labels"]
y_pred = read_examples_from_file("/content/drive/MyDrive/NER-BERT/model/pst-bc5cdr-pdr/biobert/test_predictions.txt")["labels"]

In [ ]:
from seqeval.metrics import classification_report as classification_report_seqeval

print(classification_report_seqeval(y_true, y_pred))

              precision    recall  f1-score   support

     Disease       0.85      0.85      0.85      1275
       Plant       0.58      0.17      0.27        87

   micro avg       0.85      0.81      0.83      1362
   macro avg       0.72      0.51      0.56      1362
weighted avg       0.84      0.81      0.81      1362



In [ ]:
import numpy as np
from sklearn.metrics import classification_report

print(classification_report(np.concatenate(y_true), np.concatenate(y_pred)))

              precision    recall  f1-score   support

   B-Disease       0.90      0.88      0.89      1275
     B-Plant       0.65      0.20      0.30        86
   I-Disease       0.83      0.85      0.84       796
     I-Plant       0.64      0.24      0.35        29
           O       0.99      0.99      0.99     21238

    accuracy                           0.98     23424
   macro avg       0.80      0.63      0.67     23424
weighted avg       0.98      0.98      0.98     23424



## **Train & Test = BC5CDR - Plantblog - S100 - Textberg - PDR**

In [ ]:
y_true = read_examples_from_file("/content/drive/MyDrive/NER-BERT/data/Multi-Source/S800-Plantblog-Textberg-BC5CDR/biobert/test.txt")["labels"]
y_pred = read_examples_from_file("/content/drive/MyDrive/NER-BERT/model/pst-bc5cdr-pdr3/biobert/test_predictions.txt")["labels"]

In [ ]:
from seqeval.metrics import classification_report as classification_report_seqeval

print(classification_report_seqeval(y_true, y_pred))

              precision    recall  f1-score   support

     Disease       0.86      0.86      0.86      1275
       Plant       0.82      0.84      0.83       374

   micro avg       0.85      0.86      0.85      1649
   macro avg       0.84      0.85      0.85      1649
weighted avg       0.85      0.86      0.85      1649



In [ ]:
import numpy as np
from sklearn.metrics import classification_report

print(classification_report(np.concatenate(y_true), np.concatenate(y_pred)))

              precision    recall  f1-score   support

   B-Disease       0.89      0.88      0.89      1275
     B-Plant       0.85      0.87      0.86       371
   I-Disease       0.84      0.86      0.85       798
     I-Plant       0.89      0.92      0.90       251
           O       0.99      0.99      0.99     24753

    accuracy                           0.98     27448
   macro avg       0.89      0.90      0.90     27448
weighted avg       0.98      0.98      0.98     27448



## **Train & Test = BC5CDR - Plantblog - S100 - Textberg - PDR | epoch = 10**

In [ ]:
y_true = read_examples_from_file("/content/drive/MyDrive/NER-BERT/data/Multi-Source/S800-Plantblog-Textberg-BC5CDR/biobert/test.txt")["labels"]
y_pred = read_examples_from_file("/content/drive/MyDrive/NER-BERT/model/pst-bc5cdr-pdr-e10/biobert/test_predictions.txt")["labels"]

In [ ]:
from seqeval.metrics import classification_report as classification_report_seqeval

print(classification_report_seqeval(y_true, y_pred))

              precision    recall  f1-score   support

     Disease       0.86      0.88      0.87      1275
       Plant       0.86      0.85      0.86       374

   micro avg       0.86      0.87      0.86      1649
   macro avg       0.86      0.86      0.86      1649
weighted avg       0.86      0.87      0.86      1649



In [ ]:
import numpy as np
from sklearn.metrics import classification_report

print(classification_report(np.concatenate(y_true), np.concatenate(y_pred)))

              precision    recall  f1-score   support

   B-Disease       0.89      0.90      0.89      1275
     B-Plant       0.88      0.87      0.88       371
   I-Disease       0.85      0.86      0.85       798
     I-Plant       0.88      0.94      0.91       251
           O       0.99      0.99      0.99     24753

    accuracy                           0.98     27448
   macro avg       0.90      0.91      0.90     27448
weighted avg       0.98      0.98      0.98     27448



# **Plantblog, S800, Textblog, BC5DR**

In [ ]:
y_true = read_examples_from_file("/content/drive/MyDrive/NER-BERT/data/Multi-Source/S800-Plantblog-Textberg-BC5CDR/biobert/test.txt")["labels"]
y_pred = read_examples_from_file("/content/drive/MyDrive/NER-BERT/model/pst-bc5cdr-new/biobert/test_predictions.txt")["labels"]

In [ ]:
from seqeval.metrics import classification_report as classification_report_seqeval

print(classification_report_seqeval(y_true, y_pred))

              precision    recall  f1-score   support

     Disease       0.87      0.88      0.87      1173
       Plant       0.85      0.89      0.87       288

   micro avg       0.86      0.88      0.87      1461
   macro avg       0.86      0.88      0.87      1461
weighted avg       0.86      0.88      0.87      1461



In [ ]:
import numpy as np
from sklearn.metrics import classification_report

print(classification_report(np.concatenate(y_true), np.concatenate(y_pred)))

              precision    recall  f1-score   support

   B-Disease       0.90      0.89      0.89      1173
     B-Plant       0.89      0.91      0.90       286
   I-Disease       0.84      0.86      0.85       743
     I-Plant       0.89      0.94      0.91       222
           O       0.99      0.99      0.99     23439

    accuracy                           0.98     25863
   macro avg       0.90      0.92      0.91     25863
weighted avg       0.98      0.98      0.98     25863



# **Uji Kinerja Model**

In [ ]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/NER-BERT/model/pst-bc5cdr-pdr3/biobert")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/NER-BERT/model/pst-bc5cdr-pdr3/biobert")

ner_model = pipeline('ner', model=model, tokenizer=tokenizer)

In [ ]:
sequence = "Thymus quinquecostatus Celak. (Korean name: bak-ri-hyang) is an important medicinal and aromatic herb in Korea, which is named for the spread of its fragrance over a distance of approximately 40 km. In traditional Korean systems of medicine, T. quinquecostatus has been used to treat cancer, constipation, hepatic disease, arteriosclerosis, poor circulation in women, constipation, and menstrual irregularities."
#ner_model(sequence)

In [ ]:
# Get the NER results
ner_results = ner_model(sequence)

# Post-process the NER results to concatenate subwords correctly
decoded_sequence = tokenizer.decode(tokenizer.encode(sequence, add_special_tokens=False))

final_results = []
current_entity = None

for entity in ner_results:
    if '##' in entity['word']:
        # Concatenate the subword with the previous one
        current_entity['word'] += entity['word'].replace('##', '')
        current_entity['end'] = entity['end']
    else:
        # Save the current entity and update it
        if current_entity:
            final_results.append(current_entity)
        current_entity = entity
        current_entity['word'] = entity['word']
        current_entity['start'] = decoded_sequence.find(entity['word'], entity['start'])
        current_entity['end'] = current_entity['start'] + len(current_entity['word'])

# Append the last entity
if current_entity:
    final_results.append(current_entity)

# Filter out entities with '##' in the word
final_results = [entity for entity in final_results if '##' not in entity['word']]

# Print the modified output
print(final_results)

[{'entity': 'B-Plant', 'score': 0.9979322, 'index': 1, 'word': 'Thymus', 'start': 0, 'end': 6}, {'entity': 'I-Plant', 'score': 0.9971956, 'index': 4, 'word': 'quinquecostatus', 'start': 7, 'end': 22}, {'entity': 'B-Plant', 'score': 0.99751306, 'index': 63, 'word': 'T', 'start': 249, 'end': 250}, {'entity': 'I-Plant', 'score': 0.9961617, 'index': 65, 'word': 'quinquecostatus', 'start': 252, 'end': 260}, {'entity': 'B-Disease', 'score': 0.9712861, 'index': 83, 'word': 'hepatic', 'start': 313, 'end': 313}, {'entity': 'I-Disease', 'score': 0.9907659, 'index': 86, 'word': 'disease', 'start': 321, 'end': 328}, {'entity': 'B-Disease', 'score': 0.94425815, 'index': 88, 'word': 'arteriosclerosis', 'start': 330, 'end': 339}, {'entity': 'B-Disease', 'score': 0.71015173, 'index': 99, 'word': 'constation', 'start': 375, 'end': 380}, {'entity': 'B-Disease', 'score': 0.9156232, 'index': 105, 'word': 'menstrual', 'start': 393, 'end': 395}, {'entity': 'I-Disease', 'score': 0.98608786, 'index': 109, 'wo

In [ ]:
ner_model(sequence)

[{'entity': 'B-Plant',
  'score': 0.9979322,
  'index': 1,
  'word': 'T',
  'start': 0,
  'end': 1},
 {'entity': 'B-Plant',
  'score': 0.98963493,
  'index': 2,
  'word': '##hy',
  'start': 1,
  'end': 3},
 {'entity': 'I-Plant',
  'score': 0.97208077,
  'index': 3,
  'word': '##mus',
  'start': 3,
  'end': 6},
 {'entity': 'I-Plant',
  'score': 0.9971956,
  'index': 4,
  'word': 'q',
  'start': 7,
  'end': 8},
 {'entity': 'I-Plant',
  'score': 0.99108326,
  'index': 5,
  'word': '##uin',
  'start': 8,
  'end': 11},
 {'entity': 'I-Plant',
  'score': 0.991896,
  'index': 6,
  'word': '##que',
  'start': 11,
  'end': 14},
 {'entity': 'I-Plant',
  'score': 0.99504316,
  'index': 7,
  'word': '##cos',
  'start': 14,
  'end': 17},
 {'entity': 'I-Plant',
  'score': 0.9909582,
  'index': 8,
  'word': '##tat',
  'start': 17,
  'end': 20},
 {'entity': 'I-Plant',
  'score': 0.9401694,
  'index': 9,
  'word': '##us',
  'start': 20,
  'end': 22},
 {'entity': 'B-Plant',
  'score': 0.99751306,
  'inde

## **Mode BioBERT**

In [ ]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

# Load the model and tokenizer
model_path = "dmis-lab/biobert-v1.1"
model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Create the NER pipeline
ner_model = pipeline('ner', model=model, tokenizer=tokenizer)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
sequence = "Studies are exploring if large amounts of ginger may affect insulin and lower blood sugar, so until more is known, people with diabetes can enjoy normal quantities of ginger in food but should steer clear of large-dose ginger supplements"
#ner_model(sequence)

In [ ]:
# Get the NER results
ner_results = ner_model(sequence)

# Post-process the NER results to concatenate subwords correctly
decoded_sequence = tokenizer.decode(tokenizer.encode(sequence, add_special_tokens=False))

final_results = []
current_entity = None

for entity in ner_results:
    if '##' in entity['word']:
        # Concatenate the subword with the previous one
        current_entity['word'] += entity['word'].replace('##', '')
        current_entity['end'] = entity['end']
    else:
        # Save the current entity and update it
        if current_entity:
            final_results.append(current_entity)
        current_entity = entity
        current_entity['word'] = entity['word']
        current_entity['start'] = decoded_sequence.find(entity['word'], entity['start'])
        current_entity['end'] = current_entity['start'] + len(current_entity['word'])

# Append the last entity
if current_entity:
    final_results.append(current_entity)

# Filter out entities with '##' in the word
final_results = [entity for entity in final_results if '##' not in entity['word']]

# Print the modified output
print(final_results)

[{'entity': 'LABEL_1', 'score': 0.5114185, 'index': 1, 'word': 'Studies', 'start': 0, 'end': 7}, {'entity': 'LABEL_0', 'score': 0.5294811, 'index': 2, 'word': 'are', 'start': 8, 'end': 11}, {'entity': 'LABEL_0', 'score': 0.5259547, 'index': 3, 'word': 'exploring', 'start': 12, 'end': 21}, {'entity': 'LABEL_1', 'score': 0.50698847, 'index': 4, 'word': 'if', 'start': 22, 'end': 24}, {'entity': 'LABEL_0', 'score': 0.52655965, 'index': 5, 'word': 'large', 'start': 25, 'end': 30}, {'entity': 'LABEL_1', 'score': 0.53511465, 'index': 6, 'word': 'amounts', 'start': 31, 'end': 38}, {'entity': 'LABEL_1', 'score': 0.5049051, 'index': 7, 'word': 'of', 'start': 39, 'end': 41}, {'entity': 'LABEL_0', 'score': 0.53221303, 'index': 8, 'word': 'ginger', 'start': 42, 'end': 48}, {'entity': 'LABEL_0', 'score': 0.516136, 'index': 10, 'word': 'may', 'start': 49, 'end': 52}, {'entity': 'LABEL_0', 'score': 0.51707, 'index': 11, 'word': 'affect', 'start': 53, 'end': 59}, {'entity': 'LABEL_1', 'score': 0.534078